In [7]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
from wilcoxon.spiderman import *
from wilcoxon.language import *
from wilcoxon.vocab import *
import numpy as np
import chinese
from tqdm import tqdm

In [4]:
chineseV = getVocab('Chinese')

In [12]:
def chineseUnderstandability(text):

    ## converts into simplified characters
    text = trySimplified(text.replace(' ',''))

    ## creates chinese text parser
    analyser = chinese.ChineseAnalyzer()
    totalAnaL = analyser.parse(re.sub('[^\u4e00-\u9fff]+','',text))

    ## evaluates importance and understandability of each token
    chineseV['Importance'] = chineseV['Level'].replace({'HSK 1':0.4,'HSK 2':0.6,'HSK 3':0.8, 'HSK 4':1, 'HSK 5':1, 'HSK 6':0.8, 'Important':1, np.nan:0.5})
    chineseV['Understandability'] = chineseV['Level'].replace({'HSK 1':1,'HSK 2':1,'HSK 3':1, 'HSK 4':0.9, 'HSK 5':0.8, 'HSK 6':0.7, 'Important':0.8, np.nan:0.5})
    
    # converts df into dictionary for word-to-score mapping
    chineseDict = chineseV[['Word','Importance','Understandability']].drop_duplicates('Word').set_index('Word').to_dict('index')

    ## this is to find the characters which should be quite understandable
    lowhsk = chineseV[chineseV['Understandability'] > 0.8].copy()
    lowhsk['Word'] = lowhsk['Word'].apply(lambda x: list(x))
    lowhsk = lowhsk.explode('Word')[['Word','Importance','Understandability']].drop_duplicates('Word').set_index('Word').to_dict('index')
    words = chineseV['Word'].unique()

    ## 
    for word in lowhsk:
        if len(re.findall('[\u4e00-\u9fff]+',word)) > 0:
            chineseDict.update({word:lowhsk[word]})

    understandability = 0
    importance = 0
    reinforcedWords = []
    newWords = []

    for token in list(set(totalAnaL.tokens())):
        try:
            understandability += chineseDict[token]['Understandability']
            importance += chineseDict[token]['Importance']
            if chineseDict[token]['Understandability'] < 1 and chineseDict[token]['Importance'] > 0.5:
                reinforcedWords.append(token)
        except: 
            understandings = []
            for x in token:
                try: understandings.append(chineseDict[x]['Understandability'])
                except: understandings.append(0)
            minUnderstanding = min(understandings)
            if minUnderstanding < 0.8: newWords.append(token)

    understandability = int(understandability / len(list(set(totalAnaL.tokens()))) * 100)
    importance = int(importance / len(list(set(totalAnaL.tokens()))) * 100)
    score = importance - understandability

    return {'Understandability':understandability,
            'Importance':importance,
            'Score':score,
            'New Words':newWords,
            'Text': text}

In [13]:
chineseUnderstandability("""（早报讯）淡马锡基金会将发放超过10万瓶维生素D3保健品给低收入者、年长者以及适合服用维生素D的孕妇，协助他们增强免疫力和保持骨骼和肌肉的健康。

从3月15日至28日，持橙色和蓝色社保援助计划（CHAS）卡或接受社区援助计划的新加坡人，可到参与的Guardian, Unity或Watsons药局领取维生素D。

维生素D对于增强免疫力和保持骨骼和肌肉的健康扮演重要的角色，公众在领取维生素D前，药局内的药剂师也会检查申请者是否适合服用。

竹脚妇幼医院和善达社区保健机构（SATA CommHealth）则会分别为5000名进行产前护理的孕妇以及1500名弱势年长者提供维生素D保健品。

这款维生素D是淡马锡基金会委任德国化工公司巴斯夫（BASF）所研发，含在口中即会溶化。每瓶含100颗药丸，为一名成年人约两个月的剂量。

研究显示，虽然晒太阳能帮助人体获得维生素D，但大多数新加坡人由于习惯遮阳，并在疫情期间留在室内，因此出现缺乏维生素D的情况。  

这是淡马锡关怀基金“未雨绸缪”（Stay Prepared）计划下，帮助本地社群在疫情期间，提升紧急事故应对能力的一项措施。""")

{'Understandability': 52,
 'Importance': 54,
 'Score': 2,
 'New Words': ['竹脚',
  '妇幼医院',
  '药剂师',
  '和善',
  '早报讯',
  '淡马锡',
  '保健品',
  '未雨绸缪',
  '药丸',
  '产',
  '巴斯夫',
  '弱势',
  '溶化',
  '含在口中',
  '含颗',
  '遮阳',
  '橙色',
  '剂量'],
 'Text': '（早报讯）淡马锡基金会将发放超过10万瓶维生素D3保健品给低收入者、年长者以及适合服用维生素D的孕妇，协助他们增强免疫力和保持骨骼和肌肉的健康。\n\n从3月15日至28日，持橙色和蓝色社保援助计划（CHAS）卡或接受社区援助计划的新加坡人，可到参与的Guardian,Unity或Watsons药局领取维生素D。\n\n维生素D对于增强免疫力和保持骨骼和肌肉的健康扮演重要的角色，公众在领取维生素D前，药局内的药剂师也会检查申请者是否适合服用。\n\n竹脚妇幼医院和善达社区保健机构（SATACommHealth）则会分别为5000名进行产前护理的孕妇以及1500名弱势年长者提供维生素D保健品。\n\n这款维生素D是淡马锡基金会委任德国化工公司巴斯夫（BASF）所研发，含在口中即会溶化。每瓶含100颗药丸，为一名成年人约两个月的剂量。\n\n研究显示，虽然晒太阳能帮助人体获得维生素D，但大多数新加坡人由于习惯遮阳，并在疫情期间留在室内，因此出现缺乏维生素D的情况。\n\n这是淡马锡关怀基金“未雨绸缪”（StayPrepared）计划下，帮助本地社群在疫情期间，提升紧急事故应对能力的一项措施。'}